In [9]:
from flask import Flask, request, render_template_string, redirect, url_for, render_template
from flask import Flask, render_template, request, redirect, url_for, session
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt
from io import BytesIO
import base64
from datetime import datetime
import import_ipynb
import current_tournament_leaderboard
import current_tournament_information
import bcrypt
import os

app = Flask(__name__)
app.secret_key = os.environ.get('FLASK_SECRET_KEY', 'penguin123crazy1--')
app.config['SESSION_PERMANENT'] = False
app.config['SESSION_TYPE'] = 'filesystem'

db_creds = pd.read_csv('/users/zack burnside/desktop/database_creds.csv')

def authenticate_user(username, password):
    try:
        conn = get_db_connection()
        cur = conn.cursor()

        query = """
            SELECT password
            FROM public.user_logins
            WHERE username = %s
        """
        cur.execute(query, (username,))
        result = cur.fetchone()

        if result and result[0] == password:
            cur.close()
            conn.close()
            return True
        else:
            cur.close()
            conn.close()
            return False

    except Exception as e:
        print(f"Error during authentication: {e}")
        return False

def get_db_connection():
    conn = psycopg2.connect(
        dbname=db_creds['dbname'][0],
        user=db_creds['user'][0],
        password=db_creds['password'][0],
        host=db_creds['host'][0],
        port=db_creds['port'][0]
    )
    return conn

def fetch_user_results(username=None):
    conn = get_db_connection()
    cur = conn.cursor()
    if username:
        cur.execute("SELECT user_name, player_pick, prize_money, tournament_name FROM public.weekly_tournament_results WHERE user_name = %s", (username,))
    else:
        cur.execute("SELECT user_name, player_pick, prize_money, tournament_name FROM public.weekly_tournament_results")
    results = cur.fetchall()
    cur.close()
    conn.close()
    return results

def fetch_tournament_results(tournament_name):
    conn = get_db_connection()
    cur = conn.cursor()
    cur.execute("""
        SELECT user_name, player_pick, prize_money 
        FROM public.weekly_tournament_results 
        WHERE tournament_name = %s
    """, (tournament_name,))
    results = cur.fetchall()
    cur.close()
    conn.close()
    return results

def insert_pick(username, tournament, pick):
    conn = get_db_connection()

    cur = conn.cursor()
    cur.execute("""
        INSERT INTO public.weekly_tournament_results (user_name, tournament_name, player_pick)
        VALUES (%s, %s, %s)
    """, (username, tournament, pick))
    conn.commit()
    cur.close()
    conn.close()

def fetch_stats_data():
    conn = get_db_connection()
    cursor = conn.cursor()
    cursor.execute("SELECT player_pick, COUNT(*) as pick_count FROM public.user_picks GROUP BY player_pick ORDER BY pick_count DESC")
    data = cursor.fetchall()
    conn.close()
    return data

def create_bar_chart(data):
    golfers = [row[0] for row in data]
    pick_count = [row[1] for row in data]

    fig, ax = plt.subplots(figsize=(20, 12))
    bars = ax.bar(golfers, pick_count, color='skyblue')

    for bar in bars:
        count_value = bar.get_height()
        ax.text(bar.get_x() + bar.get_width() / 2, bar.get_height(), str(count_value), ha='center', va='bottom', fontsize=8)

    ax.set_title('Most Picked Golfer', fontsize=12)
    ax.set_xlabel('Golfer Name', fontsize=10)
    ax.set_ylabel('Pick Count', fontsize=10)
    plt.xticks(rotation=45, ha='right', fontsize=8)
    plt.grid(axis='y', linestyle='--')
    plt.tight_layout()

    # Convert plot to PNG image and encode it in base64
    buffer = BytesIO()
    plt.savefig(buffer, format='png')
    buffer.seek(0)
    plot_data = buffer.getvalue()
    buffer.close()
    plot_data = base64.b64encode(plot_data).decode('utf-8')
    plt.close()

    return plot_data

def insert_pick(username, tournament, pick):
    conn = get_db_connection()
    cursor = conn.cursor()
    cursor.execute(
        "INSERT INTO user_picks (user_name, tournament_name, player_pick) VALUES (%s, %s, %s)",
        (username, tournament, pick)
    )
    conn.commit()
    cursor.close()
    conn.close()

def fetch_leaderboard():
    conn = get_db_connection()
    cursor = conn.cursor()
    cursor.execute("""
        SELECT user_name AS username, SUM(prize_money) AS total_prize_money
        FROM public.weekly_tournament_results
        GROUP BY user_name
        ORDER BY total_prize_money DESC
    """)
    return cursor.fetchall()

@app.route('/login', methods=['GET', 'POST'])
def login():
    session.clear()  # Clear the session data
    
    if request.method == 'POST':
        username = request.form['username']
        password = request.form['password']

        if authenticate_user(username, password):
            session['username'] = username
            session.permanent = False  # Session expires when the browser is closed
            return redirect(url_for('home'))
        else:
            return render_template('login.html', error='Invalid username or password')

    return render_template('login.html')

@app.route('/logout', methods=['POST'])
def logout():
    session.pop('username', None)
    return redirect(url_for('login.html'))

@app.route('/results', methods=['GET', 'POST'])
def results():
    selected_user = None
    if request.method == 'POST':
        selected_user = request.form['username']
    
    results_data = fetch_user_results(selected_user)
    usernames = [row[0] for row in fetch_user_results()]  # Fetch usernames for the dropdown
    
    return render_template_string('results.html', results_data=results_data, usernames=set(usernames), selected_user=selected_user)

@app.route('/')
def home():
    # Redirect to login if not authenticated
    if 'username' not in session:
        return redirect(url_for('login'))
    
    conn = get_db_connection()
    cursor = conn.cursor()

    def total_money_won():
        cursor.execute("SELECT SUM(prize_money) FROM public.weekly_tournament_results;")
        total_money_ = cursor.fetchone()
        return total_money_[0] if total_money_ else 0

    def total_money_left():
        cursor.execute("SELECT SUM(purse::numeric) AS total_purse FROM public.tournament_info WHERE dates::date > CURRENT_DATE;")
        total_money = cursor.fetchone()
        return total_money[0] if total_money else 0

    def fetch_leaderboard():
        cursor.execute("""
            SELECT user_name AS username, SUM(prize_money) AS total_prize_money
            FROM public.weekly_tournament_results
            GROUP BY user_name
            ORDER BY total_prize_money DESC
        """)
        return cursor.fetchall()

    try:
        last_winner = 'TBD'
        next_tournament = 'TBD'
        total_moneyWon = total_money_won()
        total_moneyLeft = total_money_left()
        leaderboard_data = fetch_leaderboard()
    except Exception as e:
        print(f"Error fetching data: {e}")
        last_winner, next_tournament, total_moneyWon, total_moneyLeft, leaderboard_data = "N/A", "N/A", 0, 0, []

    cursor.close()
    conn.close()

    if request.method == 'POST':
        username = request.form['username']
        tournament = request.form['tournament']
        pick = request.form['pick']
        
        # Insert the pick into the database
        insert_pick(username, tournament, pick)
        
        return f'Thank you! Your pick is {pick}.'

    usernames = [
        "Z", "Timmy Chips", "Goob", "C", "Steve", "Sus", "DV3", "Jlo", 
        "Ty Dolla $$$", "Ryan", "Mike D", "Mando", "Eddie Buckets", 
        "Garrett", "T Sully"
    ]
    
    tournaments = [
        "WM Phoenix Open", "The Genesis Invitational", "Mexico Open at Vidanta", 
        "Cognizant Classic", "Arnold Palmer Invitational", 
        "THE PLAYERS Championship", "Valspar Championship", "Texas Children's Houston Open", 
        "The Valero Texas Open", "Masters Tournament", "RBC Heritage", "THE CJ CUP Byron Nelson", 
        "Wells Fargo Championship", "PGA Championship", "Charles Schwab Challenge", 
        "RBC Canadian Open", "the Memorial Tournament presented by Workday", "U.S. Open", 
        "Travelers Championship", "Rocket Mortgage Classic", "John Deere Classic", 
        "Genesis Scottish Open", "The Open Championship", "3M Open", "Wyndham Championship", 
        "FedEx St. Jude Championship", "BMW Championship", "TOUR Championship"
    ]

    current_tournament_info = current_tournament_information.tournament_information()

    return render_template(
        'home.html',
        last_winner=last_winner,
        next_tournament=next_tournament,
        total_moneyWon=total_moneyWon,
        usernames=usernames,
        tournaments=tournaments,
        total_moneyLeft=total_moneyLeft,
        current_tournament_info=current_tournament_info,
        leaderboard_data=leaderboard_data
    )

@app.route('/stats')
def stats_page():
    # Fetch data and create chart (example)
    stats_data = fetch_stats_data()  # Function to fetch your data
    plot_data = create_bar_chart(stats_data)  # Function to create your chart

    # Convert plot_data to base64-encoded image (example)

    # HTML content directly in the route function
    html_content = f'''
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Statistics Page</title>
    </head>
    <body>
        <h1>Most Picked Golfer</h1>

        <!-- Display the chart using base64-encoded image -->
        <img src="data:image/png;base64, {plot_data}" alt="Chart">

        <!-- Add other HTML elements as needed -->

    </body>
    </html>
    '''

    return render_template_string(html_content)

@app.route('/tournament_results/<tournament_name>')
def tournament_results(tournament_name):
    results_data = fetch_tournament_results(tournament_name)
    tournaments = [
        "WM Phoenix Open", "The Genesis Invitational", "Mexico Open at Vidanta", 
        "Cognizant Classic", "Arnold Palmer Invitational", 
        "THE PLAYERS Championship", "Valspar Championship", "Texas Children's Houston Open", 
        "The Valero Texas Open", "Masters Tournament", "RBC Heritage", "THE CJ CUP Byron Nelson", 
        "Wells Fargo Championship", "PGA Championship", "Charles Schwab Challenge", 
        "RBC Canadian Open", "the Memorial Tournament presented by Workday", "U.S. Open", 
        "Travelers Championship", "Rocket Mortgage Classic", "John Deere Classic", 
        "Genesis Scottish Open", "The Open Championship", "3M Open", "Wyndham Championship", 
        "FedEx St. Jude Championship", "BMW Championship", "TOUR Championship"
        
    ]
    return render_template('tournament_results.html', tournament_name=tournament_name, results_data=results_data, tournaments=tournaments)

if __name__ == '__main__':
    app.run(debug=False)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [22/Jul/2024 13:10:03] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Jul/2024 13:10:05] "POST /logout HTTP/1.1" 405 -


In [56]:
def create_username_graph():
    conn = get_db_connection()
    cursor = conn.cursor()
    
    # Execute the SQL query
    cursor.execute('''SELECT user_name, date, prize_money
                      FROM PUBLIC.WEEKLY_TOURNAMENT_RESULTS''')
    
    # Fetch all results from the executed SQL query
    rows = cursor.fetchall()
    
    # Fetch column names
    colnames = [desc[0] for desc in cursor.description]
    
    # Convert the results into a DataFrame
    df = pd.DataFrame(rows, columns=colnames)
    
    # Close the cursor and connection
    cursor.close()
    conn.close()
    
    # Convert 'date' column to datetime
    df['date'] = pd.to_datetime(df['date'])
    
    # Plotting
    plt.figure(figsize=(12, 8))
    
    # Group by 'user_name' and plot each user's prize money over time
    for username, group in df.groupby('user_name'):
        plt.plot(group['date'], group['prize_money'], marker='o', label=username)
    
    # Add labels and title
    plt.xlabel('Date')
    plt.ylabel('Prize Money')
    plt.title('Prize Money Over Time for Each Username')
    plt.legend(title='Username')
    plt.grid(True)
    
    # Show the plot
    plt.show()


In [87]:
import bcrypt

def hash_password(password):
    
    salt = bcrypt.gensalt()
    hashed = bcrypt.hashpw(password.encode('utf-8'), salt)
    return hashed.decode('utf-8')


conn = get_db_connection()
cursor = conn.cursor()
usernames_passwords = {
    "Z": "random_password_1",
    "Timmy Chips": "random_password_2",
    # Add other usernames and passwords here
}

# Example of storing hashed passwords in the database
for username, password in usernames_passwords.items():
    hashed_password = hash_password(password)
    # Insert into the database
    cursor.execute(
        "INSERT INTO public.user_logins (username, password) VALUES (%s, %s)",
        (username, hashed_password)
    )
conn.commit()


In [76]:
password = 'ory5C}NV;j-!'
hashed_password = bcrypt.hashpw(password.encode('utf-8'), bcrypt.gensalt())

In [89]:
def authenticate_user(username, password):
    try:
        conn = get_db_connection()
        cur = conn.cursor()

        query = """
            SELECT password
            FROM public.user_logins
            WHERE username = %s
        """
        cur.execute(query, (username,))
        result = cur.fetchone()

        if result and password:
            cur.close()
            conn.close()
            return True
        else:
            cur.close()
            conn.close()
            return False

    except Exception as e:
        print(f"Error during authentication: {e}")
        return False
    
authenticate_user('Z','ory5C}NV;j-!')

True